# Using spatial autocorrelation on CORONAVIRUS

In [3]:
!pip install --user arcgis
!pip install --user libpysal
!pip install --user esda

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
from datetime import date, timedelta
import matplotlib.pyplot as plt
import pandas
import urllib.request

class hopkins_timeseries:
    def __init__(self, states, regions, timeseries):
        self.__states = states
        self.__regions = regions
        self.__timeseries = timeseries
    
    def get(self):
        return self.__timeseries
    
    def plot(self, names, last_days=60):
        height = 2 * len(names)
        if 0 < height:
            end_index = date.today()
            start_index = end_index-timedelta(days=last_days)
            self.__timeseries[start_index:end_index][names].plot(subplots=True, figsize=(15, height))
            
    def plot_all(self, data, last_days=60):
        height = 2 * len(data.columns)
        if 0 < height:
            end_index = date.today()
            start_index = end_index-timedelta(days=last_days)
            data.loc[start_index:end_index].plot(subplots=True, figsize=(15, height))
    
    def plot_regions(self, names, last_days=60):
        aggregated_regions = self.__aggregate_regions_by_sum(names)
        self.plot_all(aggregated_regions, last_days)
    
    def plot_states(self, names, last_days=60):
        states = self.__get_states(names)
        self.plot(states, last_days)
        
    def __aggregate_regions_by_sum(self, names):
        aggregated_timeseries = pandas.DataFrame()
        for name in names:
            states_from_region = self.__get_states_from_region(name)
            if 0 < len(states_from_region):
                aggregated_timeseries[name] = self.__timeseries[states_from_region].sum(axis=1)                
            elif name in self.__timeseries.columns:
                aggregated_timeseries[name] = self.__timeseries[name]
        return aggregated_timeseries
        
    def __get_states_from_region(self, name):
        states = []
        region_indices = [index for index in range(0, len(self.__regions)) if name == self.__regions[index]]
        for region_index in region_indices:
            state = self.__states[region_index]
            if pandas.notna(state):
                states.append(state)
        return states
    
    def __get_states(self, names):
        states = []
        for name in names:
            states_from_region = self.__get_states_from_region(name)
            if 0 < len(states_from_region):
                states += states_from_region
            else:
                state_indices = [index for index in range(0, len(self.__states)) if name == self.__states[index]]
                for state_index in state_indices:
                    state = self.__states[state_index]
                    if pandas.notna(state):
                        states.append(state)
        return states
        

def query_hopkins_data():
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
    with urllib.request.urlopen(url) as stream:
        timeseries_hopkins = pandas.read_csv(stream)
        return timeseries_hopkins
        
def query_hopkins_timeseries():
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
    with urllib.request.urlopen(url) as stream:
        timeseries_hopkins = pandas.read_csv(stream)
        date_columns = timeseries_hopkins.columns[4:]
        timeseries_indexed = timeseries_hopkins[date_columns].T
        timeseries_indexed.index = pandas.to_datetime(timeseries_indexed.index)
        states = timeseries_hopkins["Province/State"]
        regions = timeseries_hopkins["Country/Region"]
        timeseries_indexed.columns = [states[index] if 0 < pandas.notna(states[index]) else regions[index] for index in range(0, len(states))]
        return hopkins_timeseries(states, regions, timeseries_indexed)

In [2]:
corona_data = query_hopkins_data()
corona_data

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,22,24,24,40,40,74,84,94,110,110
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,64,70,76,89,104,123,146,174,186,197
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,87,90,139,201,230,264,302,367,409,454
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,53,75,88,113,133,164,188,224,267,308
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,1,2,2,3,3,3,4,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,NaN,Burma,21.9162,95.9560,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,8
249,Anguilla,United Kingdom,18.2206,-63.0686,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
250,British Virgin Islands,United Kingdom,18.4207,-64.6400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
251,Turks and Caicos Islands,United Kingdom,21.6940,-71.7979,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
